In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
from sklearn import *

[nltk_data] Downloading package stopwords to C:\Users\Priyanshi
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df=pd.read_csv("training-models/amazon.csv")

In [3]:
df.head()

,id,label,tweet
0,1,1,"Good case, Excellent value."
1,2,1,Great for the jawbone.
2,3,0,Tied to charger for conversations lasting more...
3,4,1,The mic is great.
4,5,0,I have to jiggle the plug to get it to line up...


In [4]:
df['label'].value_counts()

1    500
0    499
Name: label, dtype: int64

In [5]:
def clear_tweets(text):
    text=re.sub('<[^>]*>','',text)
    emojis=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub('[\W]+',' ',text.lower()) +' '.join(emojis).replace('-','')

    return text   

In [8]:
df['cleared_tweets']=df['tweet'].apply(clear_tweets)
df.head()

,id,label,tweet,cleared_tweets
0,1,1,"Good case, Excellent value.",good case excellent value
1,2,1,Great for the jawbone.,great for the jawbone
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...
3,4,1,The mic is great.,the mic is great
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...


In [9]:
def tokenize_tweets(tweet):
    return tweet.split()

In [10]:
df['tokenized_tweets']=df['cleared_tweets'].apply(tokenize_tweets)
df.head()

,id,label,tweet,cleared_tweets,tokenized_tweets
0,1,1,"Good case, Excellent value.",good case excellent value,"[good, case, excellent, value]"
1,2,1,Great for the jawbone.,great for the jawbone,"[great, for, the, jawbone]"
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,"[tied, to, charger, for, conversations, lastin..."
3,4,1,The mic is great.,the mic is great,"[the, mic, is, great]"
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...,"[i, have, to, jiggle, the, plug, to, get, it, ..."


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Priyanshi
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [14]:
def stopword_removal(token_tweet):
    filtered_list=[]
    for word in token_tweet:
            if word.casefold() not in stop_words:
                filtered_list.append(word)
    return filtered_list

In [19]:
df['stopwords_removed']=df['tokenized_tweets'].apply(stopword_removal)
df.head()

,id,label,tweet,cleared_tweets,tokenized_tweets,stopwords_removed
0,1,1,"Good case, Excellent value.",good case excellent value,"[good, case, excellent, value]","[good, case, excellent, value]"
1,2,1,Great for the jawbone.,great for the jawbone,"[great, for, the, jawbone]","[great, jawbone]"
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,"[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, 45, mi..."
3,4,1,The mic is great.,the mic is great,"[the, mic, is, great]","[mic, great]"
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...,"[i, have, to, jiggle, the, plug, to, get, it, ...","[jiggle, plug, get, line, right, get, decent, ..."


In [20]:
stemmer = PorterStemmer()

In [26]:
def stemming_tweets(tweet):
    stemmed_words = [stemmer.stem(word) for word in tweet]
    joined_str=""
    for word in stemmed_words:
        joined_str=joined_str+" "+word
    return joined_str

In [29]:
df['stemmed_tweets']=df['stopwords_removed'].apply(stemming_tweets)
df.head()

,id,label,tweet,cleared_tweets,tokenized_tweets,stopwords_removed,stemmed_tweets
0,1,1,"Good case, Excellent value.",good case excellent value,"[good, case, excellent, value]","[good, case, excellent, value]",good case excel valu
1,2,1,Great for the jawbone.,great for the jawbone,"[great, for, the, jawbone]","[great, jawbone]",great jawbon
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,"[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, 45, mi...",tie charger convers last 45 minut major problem
3,4,1,The mic is great.,the mic is great,"[the, mic, is, great]","[mic, great]",mic great
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...,"[i, have, to, jiggle, the, plug, to, get, it, ...","[jiggle, plug, get, line, right, get, decent, ...",jiggl plug get line right get decent volum


### Trying out new method

In [149]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)




In [150]:
# tweet_list = list(df['stemmed_tweets'].apply(tokenize_tweets))
# print(type(tweet_list))
# tokens_for_model = get_tweets_for_model(tweet_list)

pos_tweet_list=list(df['stemmed_tweets'][df['label']==1].apply(tokenize_tweets))
neg_tweet_list=list(df['stemmed_tweets'][df['label']==0].apply(tokenize_tweets))

pos_tokens = get_tweets_for_model(pos_tweet_list)
neg_tokens = get_tweets_for_model(neg_tweet_list)


In [151]:
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in pos_tokens]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in neg_tokens]

dataset = positive_dataset + negative_dataset


In [152]:
import random
random.shuffle(dataset)

In [153]:
train_data = dataset[:799]
test_data = dataset[800:]

In [154]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))



Accuracy is: 0.7688442211055276


In [160]:
custom = "Happy girls are prettiest and good"
custom=clear_tweets(custom)
custom=tokenize_tweets(custom)
custom=stopword_removal(custom)
custom=stemming_tweets(custom)
custom=tokenize_tweets(custom)
print(classifier.classify(dict([token, True] for token in custom)))

Positive


In [162]:
import pickle
data={"classifier":classifier}
with open ('saved_steps_new.pk1','wb') as file:
    pickle.dump(data,file)

In [163]:
with open('saved_steps_new.pk1','rb') as file:
    data=pickle.load(file)
model_loaded=data['classifier']
custom="This is so ugly. It is so bad remove it please"
custom=clear_tweets(custom)
custom=tokenize_tweets(custom)
custom=stopword_removal(custom)
custom=stemming_tweets(custom)
custom=tokenize_tweets(custom)
print(model_loaded.classify(dict([token, True] for token in custom)))

Negative


### Yelp, Amazon, IMDB data combined

In [308]:
yelp=pd.read_csv("training-models/yelp.csv")
amazon=pd.read_csv("training-models/amazon.csv")
yelp.head()

,id,label,tweet
0,1,0,Crust is not good.
1,2,0,Not tasty and the texture was just nasty.
2,3,1,Stopped by during the late May bank holiday of...
3,4,1,The selection on the menu was great and so wer...
4,5,0,Now I am getting angry and I want my damn pho.


In [309]:
new_dataset=amazon.append(yelp)

In [310]:
imdb=pd.read_csv("training-models/imdb.csv")
imdb.head()
def convert_int(text):
    return int(text)

imdb['label']=imdb['label'].apply(convert_int)

In [311]:
new_dataset=new_dataset.append(imdb)

In [312]:
new_dataset['label'].value_counts()

new_dataset.loc[len(new_dataset.index)] = [1000, 0, 'shut up you dumb idiot'] 
new_dataset.loc[len(new_dataset.index)] = [1001, 0, 'fuck you dumb fucking idiot']
new_dataset.loc[len(new_dataset.index)] = [1002, 0, 'ew get lost and dont come here ever again']
text='you are a sweet angel'
label_new=1

new_dataset.loc[len(new_dataset.index)] = [(new_dataset.tail(1).id.values[0]+1), label_new, text]
text='looking like angel'
label_new=1

new_dataset.loc[len(new_dataset.index)] = [(new_dataset.tail(1).id.values[0]+1), label_new, text]

text='lovely like angel'
label_new=1

new_dataset.loc[len(new_dataset.index)] = [(new_dataset.tail(1).id.values[0]+1), label_new, text]
text='angelic eyes so pretty'
label_new=1

new_dataset.loc[len(new_dataset.index)] = [(new_dataset.tail(1).id.values[0]+1), label_new, text]


In [313]:
new_dataset['label'].value_counts()

1    1503
0    1501
Name: label, dtype: int64

In [314]:
new_dataset.tail()

,id,label,tweet
2999,1002,0,ew get lost and dont come here ever again
3000,1003,1,you are a sweet angel
3001,1004,1,looking like angel
3002,1005,1,lovely like angel
3003,1006,1,angelic eyes so pretty


In [315]:
new_dataset['cleared_tweets']=new_dataset['tweet'].apply(clear_tweets)

new_dataset['tokenized_tweets']=new_dataset['cleared_tweets'].apply(tokenize_tweets)

new_dataset['stopwords_removed']=new_dataset['tokenized_tweets'].apply(stopword_removal)

new_dataset['stemmed_tweets']=new_dataset['stopwords_removed'].apply(stemming_tweets)


In [316]:
# tweet_list = list(df['stemmed_tweets'].apply(tokenize_tweets))
# print(type(tweet_list))
# tokens_for_model = get_tweets_for_model(tweet_list)

pos_tweet_list=list(new_dataset['stemmed_tweets'][new_dataset['label']==1].apply(tokenize_tweets))
neg_tweet_list=list(new_dataset['stemmed_tweets'][new_dataset['label']==0].apply(tokenize_tweets))

pos_tokens = get_tweets_for_model(pos_tweet_list)
neg_tokens = get_tweets_for_model(neg_tweet_list)


In [317]:
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in pos_tokens]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in neg_tokens]

dataset = positive_dataset + negative_dataset


In [318]:

from nltk import classify
from nltk import NaiveBayesClassifier
new_classifier = NaiveBayesClassifier.train(dataset)

print("Accuracy is:", classify.accuracy(new_classifier, dataset))



Accuracy is: 0.9324234354194407


In [319]:
custom = "fuck you"
custom=clear_tweets(custom)
custom=tokenize_tweets(custom)
custom=stopword_removal(custom)
custom=stemming_tweets(custom)
custom=tokenize_tweets(custom)
print(new_classifier.classify(dict([token, True] for token in custom)))

Negative


In [320]:
import pickle
data={"classifier":new_classifier}
with open ('saved_steps.pk1','wb') as file:
    pickle.dump(data,file)

In [321]:
with open('saved_steps.pk1','rb') as file:
    data=pickle.load(file)
model_loaded=data['classifier']
custom="looking like angel"
custom=clear_tweets(custom)
custom=tokenize_tweets(custom)
custom=stopword_removal(custom)
custom=stemming_tweets(custom)
custom=tokenize_tweets(custom)
print(model_loaded.classify(dict([token, True] for token in custom)))

Positive


In [322]:
new_dataset.tail()

,id,label,tweet,cleared_tweets,tokenized_tweets,stopwords_removed,stemmed_tweets
2999,1002,0,ew get lost and dont come here ever again,ew get lost and dont come here ever again,"[ew, get, lost, and, dont, come, here, ever, a...","[ew, get, lost, dont, come, ever]",ew get lost dont come ever
3000,1003,1,you are a sweet angel,you are a sweet angel,"[you, are, a, sweet, angel]","[sweet, angel]",sweet angel
3001,1004,1,looking like angel,looking like angel,"[looking, like, angel]","[looking, like, angel]",look like angel
3002,1005,1,lovely like angel,lovely like angel,"[lovely, like, angel]","[lovely, like, angel]",love like angel
3003,1006,1,angelic eyes so pretty,angelic eyes so pretty,"[angelic, eyes, so, pretty]","[angelic, eyes, pretty]",angel eye pretti
